In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow import keras as keras
from keras import layers as layers

In [ ]:
import os, timeit
from skimage.filters import threshold_otsu
import numpy as np
from math import inf as inf

In [ ]:
from spectral.io import envi as envi
from spectral import imshow

In [ ]:
from sklearn.decomposition import IncrementalPCA

In [ ]:
import sys

In [ ]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
from sys import platform
DATA_DIRECTORY = ""
SLASH = ""
if platform == "linux" or platform == "linux2":
    DATA_DIRECTORY = "/home/nitintyagi/wheat data/BULK/"
    SLASH = "/"
elif platform == "win32":
    DATA_DIRECTORY = "D:\mvl\wheat\data\BULK\\"
    SLASH="\\"

In [ ]:
#Constants
BAND_NUMBER = 60
FILLED_AREA_RATIO = 0.9
TOTAL_IMAGE_COUNT = 8
IMAGE_COUNT = int(TOTAL_IMAGE_COUNT/4)
NUM_VARIETIES = 2

IMAGE_WIDTH = 30
IMAGE_HEIGHT = 30

In [ ]:
ACTIVATION_TYPE =  "relu"
BATCH_SIZE = 2*NUM_VARIETIES

LEARNING_RATE_BASE = 0.0001

In [ ]:
from enum import Enum

class filter_method(Enum):
    none = 0
    snv = 1
    msc = 2
    savgol = 3
    
FILTER = filter_method(0).name

# to be set if filter chosen is savgol
WINDOW = 7
ORDER = 2
DERIVATIVE = "none"

In [ ]:
from enum import Enum
 
class feature_extraction_method(Enum):
    none = 0
    pca_loading = 1
    lda = 2
    ipca = 3

FEATURE_EXTRACTION = feature_extraction_method(1).name

NUM_OF_BANDS = 3
if FEATURE_EXTRACTION == "pca_loading" or FEATURE_EXTRACTION == "ipca":
    NUM_OF_BANDS = 8
elif FEATURE_EXTRACTION == "lda":
    NUM_OF_BANDS = 3
    assert NUM_OF_BANDS <= min(NUM_VARIETIES-1,168),"NUM_OF_BANDS is greater."


REMOVE_NOISY_BANDS = True
FIRST_BAND = 15
LAST_BAND = 161

In [ ]:
def start_timer():
    print("Testing started")
    return timeit.default_timer()

def end_timer():
    return timeit.default_timer()

def show_time(tic,toc): 
    test_time = toc - tic
    print('Testing time (s) = ' + str(test_time) + '\n')

In [ ]:
# List for All varieties
VARIETIES = []
VARIETIES_CODE = {}

for name in os.listdir(DATA_DIRECTORY):
    if (name.endswith(".hdr") or name.endswith(".bil")):
        continue
    VARIETIES_CODE[name] = len(VARIETIES)
    VARIETIES.append(name)
    if len(VARIETIES)==NUM_VARIETIES:
        break

In [ ]:
def dataset_file_name(variety):
    name = "./dataset/V"+str(variety).zfill(3)+"_IC_"+str(TOTAL_IMAGE_COUNT).zfill(5)+"_FilledArea_"+str(FILLED_AREA_RATIO)+"_NumOfBands_"+str(NUM_OF_BANDS)+"_FB_"+str(FIRST_BAND)+"_LB_"+str(LAST_BAND)+"_BandNo_"+str(BAND_NUMBER)+"_ImageHeight_"+str(IMAGE_HEIGHT)+"_ImageWidth_"+str(IMAGE_WIDTH)+"_FILTER_"+str(FILTER)+"_FeatureExtraction_"+str(FEATURE_EXTRACTION)
    if REMOVE_NOISY_BANDS:
        name+="_REMOVE_NOISY_BANDS_"+str(REMOVE_NOISY_BANDS)
    if FILTER == "savgol":
        name+="_WINDOW_"+str(WINDOW)+"_ORDER_"+str(ORDER)
    return name

In [ ]:
train_dataset = []
train_dataset_label = []
test_dataset=[]
test_dataset_label = []

for idx, v in enumerate(VARIETIES):
    print("idx: ",idx)
    if idx >= NUM_VARIETIES:
        break
    train_dataset= train_dataset + np.load(dataset_file_name(v)+"_train_dataset.npy").tolist()
    train_dataset_label = train_dataset_label + np.load(dataset_file_name(v)+"_train_dataset_label.npy").tolist()
    test_dataset = test_dataset + np.load(dataset_file_name(v)+"_test_dataset.npy").tolist()
    test_dataset_label = test_dataset_label + np.load(dataset_file_name(v)+"_test_dataset_label.npy").tolist()
    
train_dataset = np.array(train_dataset)
train_dataset_label = np.array(train_dataset_label)
test_dataset = np.array(test_dataset)
test_dataset_label = np.array(test_dataset_label)

In [ ]:
from keras.models import Model
from keras.layers import Conv2D , MaxPool2D , Input , GlobalAveragePooling2D ,AveragePooling2D, Dense , Dropout ,Activation, Flatten , BatchNormalization
from keras.optimizers import Adam
from tensorflow.keras import losses,layers

In [ ]:
def normalizeDataWholeSeed(data,normalization_type='max'):
    
    if normalization_type == 'max':
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/np.max(abs(data[idx,:,:,:]))
            
    elif normalization_type == 'l2norm':
        from numpy import linalg as LA
        for idx in range(data.shape[0]):
            data[idx,:,:,:] = data[idx,:,:,:]/LA.norm(data[idx,:,:,:])       
        
    return data

In [ ]:
def ConvBatchNormalisation(prev_layer , nbr_kernels , filter_Size , strides =(1,1) , padding = 'same',activation_type='relu'):
    x = Conv2D(filters=nbr_kernels, kernel_size = filter_Size, strides=strides , padding=padding)(prev_layer)
    x = BatchNormalization(axis=3)(x)
    x = Activation(activation=activation_type)(x)
    return x

In [ ]:
def StemBlock(prev_layer):
    x = ConvBatchNormalisation(prev_layer, nbr_kernels = 32, filter_Size=(3,3) , strides=(2,2))
    x = ConvBatchNormalisation(x, nbr_kernels = 32, filter_Size=(3,3))
    x = ConvBatchNormalisation(x, nbr_kernels = 64, filter_Size=(3,3))
    x = MaxPool2D(pool_size=(3,3) , strides=(2,2)) (x)
    x = ConvBatchNormalisation(x, nbr_kernels = 80, filter_Size=(1,1))
    x = ConvBatchNormalisation(x, nbr_kernels = 192, filter_Size=(3,3))
    x = MaxPool2D(pool_size=(3,3) , strides=(2,2)) (x)
    
    return x  

In [ ]:
def InceptionBlock_A(prev_layer  , nbr_kernels):
    
    branch1 = ConvBatchNormalisation(prev_layer, nbr_kernels = 64, filter_Size = (1,1))
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels=96, filter_Size=(3,3))
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels=96, filter_Size=(3,3))
    
    branch2 = ConvBatchNormalisation(prev_layer, nbr_kernels=48, filter_Size=(1,1))
    branch2 = ConvBatchNormalisation(branch2, nbr_kernels=64, filter_Size=(3,3)) # may be 3*3
    
    branch3 = AveragePooling2D(pool_size=(3,3) , strides=(1,1) , padding='same') (prev_layer)
    branch3 = ConvBatchNormalisation(branch3, nbr_kernels = nbr_kernels, filter_Size = (1,1))
    
    branch4 = ConvBatchNormalisation(prev_layer, nbr_kernels=64, filter_Size=(1,1))
    
    output = tf.concat([branch1 , branch2 , branch3 , branch4], axis=3)
    
    return output

In [ ]:
def InceptionBlock_B(prev_layer , nbr_kernels):
    
    branch1 = ConvBatchNormalisation(prev_layer, nbr_kernels = nbr_kernels, filter_Size = (1,1))
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels = nbr_kernels, filter_Size = (7,1))
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels = nbr_kernels, filter_Size = (1,7))
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels = nbr_kernels, filter_Size = (7,1))    
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels = 192, filter_Size = (1,7))
    
    branch2 = ConvBatchNormalisation(prev_layer, nbr_kernels = nbr_kernels, filter_Size = (1,1))
    branch2 = ConvBatchNormalisation(branch2, nbr_kernels = nbr_kernels, filter_Size = (1,7))
    branch2 = ConvBatchNormalisation(branch2, nbr_kernels = 192, filter_Size = (7,1))
    
    branch3 = AveragePooling2D(pool_size=(3,3) , strides=(1,1) , padding ='same') (prev_layer)
    branch3 = ConvBatchNormalisation(branch3, nbr_kernels = 192, filter_Size = (1,1))
    
    branch4 = ConvBatchNormalisation(prev_layer, nbr_kernels = 192, filter_Size = (1,1))
    
    output = tf.concat([branch1 , branch2 , branch3 , branch4], axis = 3)
    
    return output  

In [ ]:
def InceptionBlock_C(prev_layer):
    
    branch1 = ConvBatchNormalisation(prev_layer, nbr_kernels = 448, filter_Size = (1,1))
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels = 384, filter_Size = (3,3))
    branch1_1 = ConvBatchNormalisation(branch1, nbr_kernels = 384, filter_Size = (1,3))    
    branch1_2 = ConvBatchNormalisation(branch1, nbr_kernels = 384, filter_Size = (3,1))
    branch1 = tf.concat([branch1_1 , branch1_2], axis = 3)
    
    branch2 = ConvBatchNormalisation(prev_layer, nbr_kernels = 384, filter_Size = (1,1))
    branch2_1 = ConvBatchNormalisation(branch2, nbr_kernels = 384, filter_Size = (1,3))
    branch2_2 = ConvBatchNormalisation(branch2, nbr_kernels = 384, filter_Size = (3,1))
    branch2 = tf.concat([branch2_1 , branch2_2], axis = 3)
    
    branch3 = AveragePooling2D(pool_size=(3,3) , strides=(1,1) , padding='same')(prev_layer)
    branch3 = ConvBatchNormalisation(branch3, nbr_kernels = 192, filter_Size = (1,1))
    
    branch4 = ConvBatchNormalisation(prev_layer, nbr_kernels = 320, filter_Size = (1,1))
    
    output = tf.concat([branch1 , branch2 , branch3 , branch4], axis = 3)
    
    return output

In [ ]:
def ReductionBlock_A(prev_layer):
    
    branch1 = ConvBatchNormalisation(prev_layer, nbr_kernels = 64, filter_Size = (1,1))
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels = 96, filter_Size = (3,3))
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels = 96, filter_Size = (3,3) , strides=(2,2) ) #, padding='valid'
    
    branch2 = ConvBatchNormalisation(prev_layer, nbr_kernels = 384, filter_Size=(3,3) , strides=(2,2) )
    
    branch3 = MaxPool2D(pool_size=(3,3) , strides=(2,2) , padding='same')(prev_layer)
    
    output = tf.concat([branch1 , branch2 , branch3], axis = 3)
    
    return output

In [ ]:
def ReductionBlock_B(prev_layer):
    
    branch1 = ConvBatchNormalisation(prev_layer, nbr_kernels = 192, filter_Size = (1,1))
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels = 192, filter_Size = (1,7))
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels = 192, filter_Size = (7,1))
    branch1 = ConvBatchNormalisation(branch1, nbr_kernels = 192, filter_Size = (3,3) , strides=(2,2) , padding = 'valid')
    
    branch2 = ConvBatchNormalisation(prev_layer, nbr_kernels = 192, filter_Size = (1,1) )
    branch2 = ConvBatchNormalisation(branch2, nbr_kernels = 320, filter_Size = (3,3) , strides=(2,2) , padding='valid' )

    branch3 = MaxPool2D(pool_size=(3,3) , strides=(2,2) )(prev_layer)
    
    output = tf.concat([branch1 , branch2 , branch3], axis = 3)
    
    return output

In [ ]:
def auxiliary_classifier(prev_Layer,num_classes):
    x = AveragePooling2D(pool_size=(5,5) , strides=(3,3)) (prev_Layer)
    x = ConvBatchNormalisation(x, nbr_kernels = 128, filter_Size = (1,1))
    x = Flatten()(x)
    x = Dense(units = 768, activation='relu') (x)
    x = Dropout(rate = 0.2) (x)
    x = Dense(units = 1000, activation='softmax') (x)
    my_out = Dense(num_classes, activation='softmax')(x)
    return my_out

In [ ]:
def InceptionV3(data_num_rows, data_num_cols, num_input_chans=1, num_classes=NUM_VARIETIES, activation_type='relu', dropout_rate=0.2):
    
    input_layer = Input(shape=(data_num_rows, data_num_cols, num_input_chans))
    input_tensor = layers.experimental.preprocessing.Resizing(299, 299, interpolation="bilinear", input_shape=train_dataset.shape[1:])(input_layer)

    x = StemBlock(input_tensor)
    
    x = InceptionBlock_A(prev_layer = x ,nbr_kernels = 32)
    x = InceptionBlock_A(prev_layer = x ,nbr_kernels = 64)
    x = InceptionBlock_A(prev_layer = x ,nbr_kernels = 64)
    
    x = ReductionBlock_A(prev_layer = x )
    
    x = InceptionBlock_B(prev_layer = x  , nbr_kernels = 128)
    x = InceptionBlock_B(prev_layer = x , nbr_kernels = 160)
    x = InceptionBlock_B(prev_layer = x , nbr_kernels = 160)
    x = InceptionBlock_B(prev_layer = x , nbr_kernels = 192)
    
    Aux = auxiliary_classifier(prev_Layer = x,num_classes=num_classes)
    
    x = ReductionBlock_B(prev_layer = x)
    
    x = InceptionBlock_C(prev_layer = x)
    x = InceptionBlock_C(prev_layer = x)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(units=2048, activation='relu') (x)
    x = Dropout(rate = dropout_rate) (x)
    x = Dense(units=1000, activation='softmax') (x)
    out = Dense(num_classes, activation='softmax')(x)
    
    model = Model(inputs = input_layer , outputs = [out , Aux])
    
    return model

In [ ]:
def getInceptionV3():
    learning_rate_base = LEARNING_RATE_BASE
    activation_type = ACTIVATION_TYPE
    wheat_types =  VARIETIES
    num_classes = len(wheat_types)
    dropout_rate = 0.2
    print("--------------Load Data--------------")

    x_training = np.array(train_dataset)
    labels_training = np.array(train_dataset_label)
    
    # Normalize the data
    x_training = normalizeDataWholeSeed(x_training)
    
    # Extract some information
    num_train = x_training.shape[0]
    N_spatial = x_training.shape[1:3]
    N_channel = x_training.shape[3]
    
    print("--------------Done--------------")
    
    ############ Create a model ############
    print("--------------Create a model--------------")
    
    # Generate a model
    model = InceptionV3(data_num_rows = N_spatial[0], 
                           data_num_cols = N_spatial[1],
                           num_input_chans = N_channel, 
                           num_classes = num_classes,
                           activation_type = activation_type,
                           dropout_rate = dropout_rate)

    # Compile the model
    adam_opt = Adam(learning_rate=LEARNING_RATE_BASE, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.01)
    model.compile(optimizer=adam_opt, loss=[losses.sparse_categorical_crossentropy,losses.sparse_categorical_crossentropy],loss_weights=[1, 0.3],metrics=['accuracy'])
    print("---------Completed---------")
    return model

In [ ]:
model = getInceptionV3()

In [ ]:
x_train = []
y_train = []
x_val = []
y_val = []

for i in range(len(train_dataset)):
    if i%5==0:
        x_val.append(train_dataset[i])
        y_val.append(train_dataset_label[i])
    else:
        x_train.append(train_dataset[i])
        y_train.append(train_dataset_label[i])
        
x_train = np.array(x_train)
y_train = np.array(y_train)
y_train = [y_train,y_train]

x_val = np.array(x_val)
y_val = np.array(y_val)
y_val = [y_val,y_val]

In [ ]:
model.summary()

In [ ]:
for x in range(1):
    print("From epochs: ",20*x+1," to ",20+20*x)
    tic = start_timer()
    model.fit(x_train ,y_train ,batch_size=BATCH_SIZE ,epochs=20, verbose=2, validation_data=(x_val, y_val), shuffle=True)
    toc = end_timer()
    show_time(tic,toc)
    print("for testing")
    print(model.evaluate(test_dataset,test_dataset_label))
    print("for training")
    print(model.evaluate(train_dataset,train_dataset_label))